In [6]:
import os
import pandas as pd
import numpy as np
import hashlib as hlib
import requests as r
import swifter
import sha3, binascii
import multiprocessing as mp
from ecdsa import SigningKey, SECP256k1
from cryptotools.BTC import PrivateKey, PublicKey
from time import sleep
from tqdm import tqdm, trange
from IPython.display import clear_output

coin = 'btc'

seedpath = 'seedlists/'
genpath = f'{coin}Addresses/'

interval = 0.01
def getAddressBalance(address):
    key = 'TA1B32C3Z993QCVS4KPDS36S1Y94K3BJ49'
    etherscanURL = f'https://api.etherscan.io/api?module=account&action=balance&address={address}&tag=latest&apikey={key}'
    data = r.get(etherscanURL)
    print(data.json())
    balance = float(data.json()['result']) #returns balance in wei
    balance = balance / 1000000000000000000 #convert wei to ethereum
    sleep(interval)
    return balance

# df.loc[1] = ['bruh', '01359u31901922jkfsafj21-awq', '0xAb5801a7D398351b8bE11C439e05C5B3259aeC9B']
# df.loc[2] = ['wowwie2138', '0qhjkqu31asggasssd21-ggss', '0xAb5801a7D398351b8bE11C439e05C5B3259aeC9B']
# addy = getAddress(getHash('💯'), 'eth')
# getAddressBalance(addy)

In [7]:
#create dataframe with raw password data


print(p)

ranges = (0, 100000) #10000000

filename = 'rockyou.txt'


with open(seedpath + filename, 'r',encoding='latin-1') as file:
    passwords = file.read().split('\n')[ranges[0]:ranges[1]]
    

    
df = pd.DataFrame(passwords, columns = ['raw'])

def getHash(text, hasher = 'sha256', encoder = 'utf-8'):
    if hasher == 'sha256':
        return hlib.sha256(text.encode(encoder)).hexdigest()
    if hasher == 'md5':
        return hlib.md5(text.encode(encoder)).hexdigest()
    return "ERROR"

def md5(text):
    return getHash(text, 'md5')

tqdm.pandas()
def createHashList(data, hashFunct):
    return data.swifter.apply(hashFunct)

# df['intermediate'] = createHashList(df['raw'], getHash)
df['private_key'] = createHashList(df['raw'], getHash)

def getAddress(private_key, coin = coin):
    if coin == 'eth':
        keccak = sha3.keccak_256()
        private_key = bytes(private_key, 'utf-8')
        private_key = binascii.unhexlify(private_key)
        private = SigningKey.from_string(private_key, curve=SECP256k1)
        public = private.get_verifying_key().to_string()
        keccak.update(public)
        address = "0x{}".format(keccak.hexdigest()[24:])
        return address
    elif coin == 'btc':
        keccak = sha3.keccak_256()
        private_key = bytes(private_key, 'utf-8')
        private_key = binascii.unhexlify(private_key)
        private = SigningKey.from_string(private_key, curve=SECP256k1)
        public = private.get_verifying_key().to_string("uncompressed")
        public = PublicKey.decode(public)
        #P2PKH (1) for legacy, P2SH for new address (3), P2WPKH (bc) for advanced address
        address = public.to_address('P2PKH') #
        return address
    return


df['address'] = createHashList(df['private_key'], getAddress)
df.head()

df.to_csv(f'{genpath}{coin}addresses-{filename}-{ranges[1] // 1000}K.txt', index = False)

Pandas Apply:   0%|          | 0/100000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/100000 [00:00<?, ?it/s]

KeyboardInterrupt: 